# Sharing Variables

What is wrong with the following code?

In [1]:
import tensorflow as tf
import numpy as np
from scipy.stats import norm


def fnn(x, output_dim):
    #weights and biases
    w1 = tf.Variable(tf.random_normal([10, 2], stddev=0.35), name="weights1")
    b1 = tf.Variable(tf.zeros([2]), name="biases1")

    w2 = tf.Variable(tf.random_normal([2, output_dim], stddev=0.35), name="weights2")
    b2 = tf.Variable(tf.zeros([2]), name="biases2")

    # nn operators
    y1 = tf.nn.relu(tf.matmul(x, w1) + b1)
    y2 = tf.nn.sigmoid(tf.matmul(y1,w2) + b2)
    return y2, [w1, w2]

# Defining the computational graph
x1 = tf.placeholder(tf.float32, shape=(1, 10))
y1, theta1 = fnn(x1, 1)

# The second network has different weights and biases
x2 = tf.placeholder(tf.float32, shape=(1, 10))
y2, theta2 = fnn(x2, 1)

# Initializing the session
with tf.Session() as sess:
    tf.initialize_all_variables().run()
    # Feeding and Fetching data
    theta1, theta2 = sess.run([theta1, theta2], {x1: np.random.random([1, 10]), x2: np.random.random([1, 10])})
    print("theta1", theta1[0])
    print("theta2", theta2[0])

theta1 [[ 0.44360071 -0.55896199]
 [-0.30600622 -0.5037573 ]
 [-0.53044277  0.16552085]
 [ 0.31216997  0.17150772]
 [ 0.04960573  0.1461049 ]
 [ 0.01075457  0.17645364]
 [ 0.06395692  0.14987926]
 [ 0.00446167 -0.56886625]
 [-0.07550567 -0.2945618 ]
 [ 0.39035973  0.01515578]]
theta2 [[-0.82923883 -0.39798287]
 [-0.43532887 -0.27481681]
 [-0.06774535  0.59454221]
 [ 0.2501682  -0.871499  ]
 [-0.20009017 -0.29256272]
 [-0.14484291  0.09395451]
 [ 0.20101026 -0.46657833]
 [-0.69620067 -0.72480702]
 [-0.09735443 -0.78983605]
 [ 0.19394319  0.02787798]]


1. It is hard to add a layer to the network.
2. We cannot share the weights and biases. (Each function call creates a new set of variables)

Creating networks using tf.Variable can get complicated. Instead, use tf.get_variable() to create/return variables and tf.variable_scope() to manage namespaces.

In [2]:
# function for creating nn layers
def linear(x, out_dim, name, activation_fn=None):
    with tf.variable_scope(name):
        #look for name/weights
        w = tf.get_variable(name='weights', shape=[x.get_shape()[1], out_dim], dtype=tf.float32, initializer=tf.random_normal_initializer())
        b = tf.get_variable(name='biases', shape=[out_dim], dtype=tf.float32, initializer=tf.constant_initializer(0.0))
        out = tf.matmul(x, w) + b
        if activation_fn != None:
            out = activation_fn(out)
    return out, [w, b]

# Computational Graph
with tf.variable_scope("ffn") as scope:
    x1 = tf.placeholder(tf.float32, shape=(1, 10))
    y11, theta11 = linear(x1, 10, name="h", activation_fn=tf.nn.relu)
    y12, theta12 = linear(y1, 1, name="out", activation_fn=tf.nn.sigmoid)

    #scope.reuse_variables()

    x2 = tf.placeholder(tf.float32, shape=(1, 10))
    y21, theta21 = linear(x2, 10, name="h", activation_fn=tf.nn.relu)
    y22, theta22 = linear(y1, 1, name="out", activation_fn=tf.nn.sigmoid)


# Initializing the session
with tf.Session() as sess:
    print("session")
    tf.initialize_all_variables().run()
    # Feeding and Fetching data
    theta1, theta2 = sess.run([theta12, theta22], {x1: np.random.random([1, 10]), x2: np.random.random([1, 10])})
    print(theta1[0])
    print(theta2[0])


ValueError: Variable ffn/h/weights already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "<ipython-input-2-0cf02566ad7f>", line 5, in linear
    w = tf.get_variable(name='weights', shape=[x.get_shape()[1], out_dim], dtype=tf.float32, initializer=tf.random_normal_initializer())
  File "<ipython-input-2-0cf02566ad7f>", line 15, in <module>
    y11, theta11 = linear(x1, 10, name="h", activation_fn=tf.nn.relu)
  File "/Users/aida/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2885, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)


Running the above code raises an error because the variable names are reused: "Variable h/weights already exists, disallowed. Did you mean to set reuse=True in VarScope."

If you need to share the variables, you need to explictly specify it using reuse_variables(). In the above code, you need to uncomment scope.reuse_variables().

